# Entrenamiento del modelo (Configuración 1)

* La cantidad máxima de palabras por cada sentencia es de 10.
* Batch size: 518
* Épocas: 60
* Learnig rate: 0.001

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/iA

Mounted at /gdrive
/gdrive/.shortcut-targets-by-id/1ccQ0NRVtxcnMDQHOrldbGFQDfXPWk9Jj/iA


In [2]:
%ls

20.txt
BackwardForward.ipynb
checkpoint-10ML-30ep-META-512bz-loss29-valoss41.pt
checkpoint-10ML-30ep-META-512bz-loss30-valoss41-con-tildes.pt
checkpoint-10ML-60ep-META-512bz-loss26-valoss46-con-tildes2.pt
checkpoint-15ML-20EP.pt
checkpoint-15ML-30ep-META-512bz-loss40-valoss54-con-tildes.pt
checkpoint-20ML-15ep-data-METALW-512bz-con-tildes-300em-300h.pt
checkpoint-20ML-25ep-data-METALW-512bz-con-tildes-300em-300h.pt
checkpoint-20ML-35ep-data-METALW-512bz-con-tildes-300em-300h.pt
checkpoint-20ML-50ep-data-METALW-512bz-con-tildes-300em-300h.pt
checkpoint-20ML-75ep-data-METALW-512bz-con-tildes-300em-300h.pt
checkpoint-20ML-85ep-data-METALW-512bz-con-tildes-300em-300h.pt
checkpoint-25ML-13EP-300BS.pt
checkpoint-25ML-15ep-data-METALW-512bz-con-tildes-200em-200h.pt
checkpoint-25ML-25ep-data-METALW-512bz-con-tildes-200em-200h.pt
checkpoint-25ML-35ep-data-METALW-512bz-con-tildes-200em-200h.pt
checkpoint-25ML-50ep-data-METALW-512bz-con-tildes-200em-200h.pt
checkpoint-25ML-65ep-data-METALW-512bz-

### Preparación de la data

In [3]:
import torch
from torch.jit import script
import torch.nn as nn
import random
import re
import unicodedata
from io import open
import itertools
from tqdm import tqdm


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

MAX_LENGTH = 10 # Cantidad de palabras máxima por cada sentencia

In [4]:
# Linea: pares de sentencias separadas por \t
# Sentencia: Texto de la pregunta o la respuesta.
# par: Un vector, cada vector contiene dos senticias: pregunta y la respuesta

PAD_token = 0  # Token para rellenar las sentencias con una cantidad menor a MAX_LENGTH
SOS_token = 1  # Token que indica el inicio de la sentencia
EOS_token = 2  # Token que indica el final de la sentencia

# Objeto Voc: Procesará cada sentencia de casa línea 
# Nos ayudará a generar una mapeo de cada palabra a indices (números)
# lo que permitirá obtener el índice que corresponde a cada palabra, la palabra que le
# corresponde a cada índice y la cantidad de veces que una palabra se repíte
class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {"PAD":PAD_token , "SOS":SOS_token , "EOS":EOS_token }
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Los 3 tokens inicializados SOS, EOS, PAD

    def agregarSentencia(self, sentence):
        for word in sentence.split(' '):
            self.agregarPalabra(word)

    def agregarPalabra(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def indiceDeSentencia(self, sentencia):
        return [self.word2index[word] for word in sentencia.split(' ')] + [EOS_token]

    def sentenciaDeIndice(self, indice):
        return [self.index2word[idx] for idx in indice]

    # Remueve las palabras que se repiten menos de una cierta cantidad de veces
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinicializamos los diccionarios
        self.word2index = {"PAD":PAD_token , "SOS":SOS_token , "EOS":EOS_token }
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Los 3 tokens inicializados SOS, EOS, PAD

        for word in keep_words:
            self.agregarPalabra(word)

In [5]:
# Función que normalizará cada sentencia
def unicodeToAscii(s):
    return ''.join(
        # c for c in unicodedata.normalize('NFC', s) # Para eliminar
        c for c in unicodedata.normalize('NFC', s)
        if unicodedata.category(c) != 'Mn'
    )

# Normalizamos cada sentencia
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip()) # Normalizamos y pasamos todas las letras a minúsculas
    # Pasamos cada sentencia a minúsculas, removemos los espacios y carácteres que no son letras excluyendo los números
    s = re.sub(r"([.¡!¿?])", r" \1", s)  # Mantenemos los signos interrogación y exclamación
    s = re.sub(r"[^A-zÁ-ú.¡!¿?0-9]+", r" ", s) # Mantenemos las tildes y números
    # s = re.sub(r"\¿", r"¿ ", s) # Agregamos un espacio a los signos de interrogación para que se cuente como una palabra
    # s = re.sub(r"\?", r" ?", s) # Agregamos un espacio a los signos de interrogación para que se cuente como una palabra
    # s = re.sub(r"\¡", r"¡ ", s) # Agregamos un espacio a los signos de exclamación para que se cuente como una palabra
    # s = re.sub(r"\!", r" !", s) # Agregamos un espacio a los signos de exclamación para que se cuente como una palabra
    # s = re.sub(r"\s+", r" ", s).strip() # Eliminamos los espacios demás
    # s = re.sub(r"\.", r"", s).strip() # Eliminamos los puntos

    # s = re.sub(r"([¡!¿?])", r" \1", s)
    # s = re.sub(r"[^A-z.¡!¿?0-9]+", r" ", s) # Elimina tildes
    s = re.sub(r"\.", r"", s)
    s = re.sub(r"\¿\s+", r"¿", s)
    s = re.sub(r"\s+\?", r"?", s)
    s = re.sub(r"\¡\s", r"¡", s)
    s = re.sub(r"\!\s", r"!", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s



# Leemos las lineas del archivo y devolvemos los pares y un objeto Voc
def readVocs(datafile, corpus_name):
    print("Leyendo líneas...")
    # Leemos el archivo y devuelve una lista de líneas
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Dividimos cada linea en pares, normaliza normalizando cada sentencia
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Retorna True si ambas sentencias en el par tienen una cantidad de palabras menores que MAX_LENGTH
def filtrarPar(p):
    # Las sentencias de entrada, necesitamos un espacio para el token SOS
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filtra los pares usando la función filtrarPares
def filtrarPares(pairs):
    return [pair for pair in pairs if filtrarPar(pair)]

# Usando las funciones definidas arriba generamos el diccionario que mapea de palabras a índices
# devolverá el objeto voc y la lista de pares
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Empieza la preparación de la data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filtrarPares(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Contando las palabras...")
    for pair in pairs:
        voc.agregarSentencia(pair[0])

        
        voc.agregarSentencia(pair[1])
        

    print("Cantidad total de palabras:", voc.num_words)
    
    
    return voc, pairs



save_dir = "./"
datafile = "METALW.txt"
corpus = "./"
corpus_name = "dataf_s2s"
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")

for pair in pairs[:10]:
    print(pair)


Empieza la preparación de la data ...
Leyendo líneas...
Read 319593 sentence pairs
Trimmed to 175166 sentence pairs
Contando las palabras...
Cantidad total de palabras: 40305

pairs:
['hola ¿en qué puedo ayudarle?', 'sí']
['sí', '¿cómo puedo ser de ayuda?']
['¿cómo puedo ser de ayuda?', 'quiero saber sobre la política que tengo']
['quiero saber sobre la política que tengo', 'bien ¿puedo conseguir tu nombre por favor?']
['bien ¿puedo conseguir tu nombre por favor?', '¿cubre los daños causados por el agua?']
['hola ¿en qué puedo ayudarle?', 'tengo una pregunta sobre mi política']
['tengo una pregunta sobre mi política', 'seguro ¿puedes decirme tu número de póliza?']
['seguro ¿puedes decirme tu número de póliza?', '3425512']
['3425512', 'bien tengo tu póliza aquí ¿cuál es tu pregunta?']
['bien tengo tu póliza aquí ¿cuál es tu pregunta?', 'cubre los daños causados por el agua?']


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class Dataset(torch.utils.data.Dataset):
    def __init__(self, entrada_voc, salida_voc, pares, max_length):
        self.entrada_voc = entrada_voc
        self.salida_voc = salida_voc
        self.pares = pares
        self.max_length = max_length
    
    def __len__(self):
        return len(self.pares)
        
    def __getitem__(self, ix):        
        entrada = torch.tensor(self.entrada_voc.indiceDeSentencia(self.pares[ix][0]), device=device, dtype=torch.long)
        salida = torch.tensor(self.salida_voc.indiceDeSentencia(self.pares[ix][1]), device=device, dtype=torch.long)
        # metemos padding a todas las frases hast a la longitud máxima
        return torch.nn.functional.pad(entrada, (0, self.max_length - len(entrada)), 'constant', self.entrada_voc.word2index['PAD']), \
            torch.nn.functional.pad(salida, (0, self.max_length - len(salida)), 'constant', self.salida_voc.word2index['PAD'])



## División de la data

In [7]:
df = pd.DataFrame(pairs, columns =['Pregunta', 'Respuesta'])

X_train, X_test, y_train, y_test = train_test_split(df['Pregunta'], df['Respuesta'], test_size=0.20, random_state=42)

train = pd.concat([X_train, y_train], axis=1).values.tolist()
test = pd.concat([X_test, y_test], axis=1).values.tolist()

In [8]:
dataset = {
    'train': Dataset(voc, voc, train, max_length=MAX_LENGTH),
    'test': Dataset(voc, voc, test, max_length=MAX_LENGTH)
}

len(dataset['train']), len(dataset['test'])

(140132, 35034)

In [9]:
entrada, salida = dataset['test'][150]
entrada.shape, salida.shape

(torch.Size([10]), torch.Size([10]))

In [ ]:
train

In [10]:
voc.sentenciaDeIndice(entrada.tolist()), voc.sentenciaDeIndice(salida.tolist())

(['hola',
  '¿en',
  'qué',
  'puedo',
  'ayudarle?',
  'EOS',
  'PAD',
  'PAD',
  'PAD',
  'PAD'],
 ['necesito',
  'saber',
  'si',
  'dejé',
  'las',
  'puertas',
  'abiertas',
  'EOS',
  'PAD',
  'PAD'])

## Modelo

In [11]:
class Encoder(torch.nn.Module):
  def __init__(self, longitud_entrada, longitud_embedding=100, longitud_oculta=100, n_capas=2):
    super().__init__()
    self.longitud_oculta = longitud_oculta
    self.embedding = torch.nn.Embedding(longitud_entrada, longitud_embedding)
    self.gru = torch.nn.GRU(longitud_embedding, longitud_oculta, num_layers=n_capas, batch_first=True)

  def forward(self, oraciones_entrada):
    embedded = self.embedding(oraciones_entrada)
    salidas, oculta = self.gru(embedded)
    return salidas, oculta

In [12]:
class AtencionDecoder(torch.nn.Module):
  def __init__(self, longitud_entrada, longitud_embedding=100, longitud_oculta=100, n_layers=2, longitud_maxima=MAX_LENGTH):
    super().__init__()
    self.embedding = torch.nn.Embedding(longitud_entrada, longitud_embedding)
    self.gru = torch.nn.GRU(longitud_embedding, longitud_oculta, num_layers=n_layers, batch_first=True)
    self.out = torch.nn.Linear(longitud_oculta, longitud_entrada)

    self.atencion = torch.nn.Linear(longitud_oculta + longitud_embedding, longitud_maxima)
    self.combinar_atencion = torch.nn.Linear(longitud_oculta * 2, longitud_oculta)

  def forward(self, palabras_entrada, oculta, salidas_encoder):
    embedded = self.embedding(palabras_entrada)
    pesos_atencion = torch.nn.functional.softmax(self.atencion(torch.cat((embedded.squeeze(1), oculta[0]), 1)))
    atencion_aplicada = torch.bmm(pesos_atencion.unsqueeze(1), salidas_encoder)
    salida = torch.cat((embedded.squeeze(1), atencion_aplicada.squeeze(1)), 1)
    salida = self.combinar_atencion(salida)
    salida = torch.nn.functional.relu(salida)
    salida, oculta = self.gru(salida.unsqueeze(1), oculta)
    salida = self.out(salida.squeeze(1))
    return salida, oculta, pesos_atencion

In [13]:
voc.num_words

40305

In [14]:
encoder = Encoder(longitud_entrada=voc.num_words)
decoder = AtencionDecoder(longitud_entrada=voc.num_words)

## Entrenamiento

In [15]:
dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=512, shuffle=True),
    'test': torch.utils.data.DataLoader(dataset['test'], batch_size=512, shuffle=False),
}

entrada, salida = next(iter(dataloader['test']))
entrada.shape, salida.shape

(torch.Size([512, 10]), torch.Size([512, 10]))

In [16]:
def entrenamiento(encoder, decoder, epochs=10):
    encoder.to(device)
    decoder.to(device)
    encoder_optimizador = torch.optim.Adam(encoder.parameters(), lr=1e-3)
    decoder_optimizador = torch.optim.Adam(decoder.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        encoder.train()
        decoder.train()
        train_perdida = []

        batches = tqdm(dataloader['train'])
        
        for batch in batches:
            entrada_sentencia, salida_sentencia = batch
            base = entrada_sentencia.shape[0]
            perdida = 0
            encoder_optimizador.zero_grad()
            decoder_optimizador.zero_grad()
            # obteniendo el último estado oculto del encoder
            encoder_salida, oculta = encoder(entrada_sentencia)
            # calculando las salidas del decoder de manera recurrente
            decoder_entrada = torch.tensor([[voc.word2index['SOS']] for b in range(base)], device=device)
            for i in range(salida_sentencia.shape[1]):
                salida, oculta, peso_atencion = decoder(decoder_entrada, oculta, encoder_salida)
                perdida += criterion(salida, salida_sentencia[:, i].view(base))
                # el siguiente input será la palabra predicha
                decoder_entrada = torch.argmax(salida, axis=1).view(base, 1)
            # optimizacion
            perdida.backward()
            encoder_optimizador.step()
            decoder_optimizador.step()
            train_perdida.append(perdida.item())
            batches.set_description(f'Epoch {epoch}/{epochs} loss {np.mean(train_perdida):.5f}')
        
        
        val_loss = []
        encoder.eval()
        decoder.eval()
        with torch.no_grad():

            batches = tqdm(dataloader['test'])

            for batch in batches:
                entrada_sentencia , salida_sentencia = batch
                base = entrada_sentencia.shape[0]
                perdida = 0
                #obtenemos el último estado oculto del encoder
                encoder_salida, oculta = encoder(entrada_sentencia)
                # calculando las salidas del decoder de manera recurrente
                decoder_entrada = torch.tensor([[voc.word2index['SOS']] for b in range(base)], device=device)
                for i in range(salida_sentencia.shape[1]):
                    salida, oculta, peso_atencion = decoder(decoder_entrada, oculta, encoder_salida)
                    perdida += criterion(salida, salida_sentencia[:, i].view(base))
                    # el siguiente input será la palabra predicha
                    decoder_entrada = torch.argmax(salida, axis=1).view(base, 1)
                val_loss.append(perdida.item())
                batches.set_description(f'Epoch {epoch}/{epoch} val_loss {np.mean(val_loss):.5f}')
            

In [ ]:
entrenamiento(encoder, decoder, epochs=30)

In [ ]:
entrenamiento(encoder, decoder, epochs=30)

  0%|          | 0/274 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
Epoch 30/30 val_loss 41.44170: 100%|██████████| 69/69 [00:06<00:00, 10.65it/s]


# Guardar modelo

In [ ]:
torch.save({'encoder':encoder.state_dict(),
            'decoder':decoder.state_dict()},
            'checkpoint-10ML-60ep-META-512bz-loss26-valoss46-con-tildes2.pt')

# Cargar modelo

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/iA

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/iA


In [ ]:
%ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
20.txt
BackwardForward.ipynb
checkpoint-10ML-30ep-META-512bz-loss29-valoss41.pt
checkpoint-10ML-30ep-META-512bz-loss30-valoss41-con-tildes.pt
checkpoint-15ML-20EP.pt
checkpoint-15ML-30ep-META-512bz-loss40-valoss54-con-tildes.pt
checkpoint2.pt
checkpoint-50ML-20EP.pt
checkpoint.pt
continnuacion/
cultura_general_1
data_freider.txt
Data_PC2/
final.txt
GRL_Book.pdf
METALW.txt


In [17]:
encoder = Encoder(longitud_entrada=voc.num_words).cuda()
decoder = AtencionDecoder(longitud_entrada=voc.num_words).cuda()
# checkpoint = torch.load('checkpoint.pt')
checkpoint = torch.load('checkpoint-10ML-30ep-META-512bz-loss30-valoss41-con-tildes.pt')
encoder.load_state_dict(checkpoint['encoder'])
decoder.load_state_dict(checkpoint['decoder'])


<All keys matched successfully>

In [18]:
def zeroPadding(l, fillvalue=PAD_token):
     return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [19]:
def inputVar(l, voc):
     #print(l)
     print(l)
     indexes_batch = [voc.word2index[sentence] for sentence in l.strip().split(' ')] + [EOS_token]
     print(indexes_batch)
     #lengths = torch.tensor(indexes_batch, device=device, dtype=torch.long)
     #print(lengths)
     #padList = zeroPadding(indexes_batch)
     padVar = torch.tensor(indexes_batch, device=device, dtype=torch.long)
     return torch.nn.functional.pad(padVar,(0,MAX_LENGTH-len(padVar)),'constant',voc.word2index['PAD'])  

In [20]:
def predict(input_sentence):
    # obtenemos el último estado oculto del encoder
    print("encoder")
    encoder_outputs, hidden = encoder(input_sentence.unsqueeze(0))
    # calculamos las salidas del decoder de manera recurrente
    print("decoder")
    decoder_input = torch.tensor([[voc.word2index['SOS']]], device=device)
    # iteramos hasta que el decoder nos de el token <eos>
    outputs = []
    decoder_attentions = torch.zeros(MAX_LENGTH, MAX_LENGTH)
    i = 0
    while True and i<MAX_LENGTH:
        output, hidden, attn_weights = decoder(decoder_input, hidden, encoder_outputs)
        decoder_attentions[i] = attn_weights.data
        i += 1
        decoder_input = torch.argmax(output, axis=1).view(1, 1)
        outputs.append(decoder_input.cpu().item())
        if decoder_input.item() == voc.word2index['EOS']:
            break
    return voc.sentenciaDeIndice(outputs), decoder_attentions

* ¿quieres helado? Sí
* ¿cómo te llamas? mi nombre
* ¿hola? hola puedo
* hola hola
* ¿Como estás? estoy
* ¿cuántos años tienes? 8
* ¿estás bien? si si

* ¿cómo estás? estoy bien
* cuántos años tienes? 122 años
* hola hola
* ¿cómo te llamas? mi nombre
* tienes hambre? sí

In [34]:
salidaTextual, salidaCodificada = predict(inputVar("tienes hambre?",voc))
print(salidaTextual)

tienes hambre?
[106, 21930, 2]
encoder
decoder
['sí', 'EOS']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


In [32]:
print(salidaTextual)

['no', 'EOS']
